In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression


In [2]:
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [3]:
import os
os.getcwd()


'/home/intellact/Documents/genDs/Daily Assignment'

In [7]:
df = pd.read_csv(r'/home/intellact/Documents/genDs/Daily Assignment/coffee_shop_revenue.csv')
print(f"✓ Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")
print(df.head(10))


✓ Dataset loaded successfully!
Dataset shape: (2000, 7)
   Number_of_Customers_Per_Day  Average_Order_Value  Operating_Hours_Per_Day  \
0                          152                 6.74                       14   
1                          485                 4.50                       12   
2                          398                 9.09                        6   
3                          320                 8.48                       17   
4                          156                 7.44                       17   
5                          121                 8.88                        6   
6                          238                 9.00                       11   
7                           70                 7.81                       10   
8                          152                 8.78                       14   
9                          171                 7.73                        7   

   Number_of_Employees  Marketing_Spend_Per_Day  Location_Foot_

In [8]:
print("\n📈 Dataset Info:")
print(df.info())


📈 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Number_of_Customers_Per_Day  2000 non-null   int64  
 1   Average_Order_Value          2000 non-null   float64
 2   Operating_Hours_Per_Day      2000 non-null   int64  
 3   Number_of_Employees          2000 non-null   int64  
 4   Marketing_Spend_Per_Day      2000 non-null   float64
 5   Location_Foot_Traffic        2000 non-null   int64  
 6   Daily_Revenue                2000 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 109.5 KB
None


In [9]:
#Check for missing values
print(f"\n🔍 Missing values: {df.isnull().sum()}")


🔍 Missing values: Number_of_Customers_Per_Day    0
Average_Order_Value            0
Operating_Hours_Per_Day        0
Number_of_Employees            0
Marketing_Spend_Per_Day        0
Location_Foot_Traffic          0
Daily_Revenue                  0
dtype: int64


In [13]:
X = df.drop("Daily_Revenue", axis=1)
y = df["Daily_Revenue"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

print(f"\n📊 Data Split:")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")


📊 Data Split:
Training set: 1400 samples
Test set: 600 samples


In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
selector = SelectKBest(score_func=f_regression,k='all')  # Select top 15 features
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

selected_features = X.columns[selector.get_support()]
print(f"\n🎯 Feature Selection: Top {len(selected_features)} features selected")
print("Selected features:", list(selected_features))


🎯 Feature Selection: Top 6 features selected
Selected features: ['Number_of_Customers_Per_Day', 'Average_Order_Value', 'Operating_Hours_Per_Day', 'Number_of_Employees', 'Marketing_Spend_Per_Day', 'Location_Foot_Traffic']


In [18]:
print("\n🤖 Training Linear Regression Model:")
model = LinearRegression()
model.fit(X_train_selected, y_train)
print("✓ Model trained successfully")


🤖 Training Linear Regression Model:
✓ Model trained successfully


In [21]:
# Make predictions
y_pred_train = model.predict(X_train_selected)
y_pred_test = model.predict(X_test_selected)
# Calculate comprehensive metrics
def calculate_metrics(y_true, y_pred, dataset_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Additional metrics
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n📊 {dataset_name} Set Performance:")
    print(f"R² Score: {r2:.4f}")
    print(f"RMSE: ${rmse:.2f}")
    print(f"MAE: ${mae:.2f}")
    print(f"MAPE: {mape:.2f}%")
    
    return {'R2': r2, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

In [22]:
# Evaluate model performance
train_metrics = calculate_metrics(y_train, y_pred_train, "Training")
test_metrics = calculate_metrics(y_test, y_pred_test, "Test")


📊 Training Set Performance:
R² Score: 0.8908
RMSE: $323.02
MAE: $254.45
MAPE: 19.65%

📊 Test Set Performance:
R² Score: 0.8927
RMSE: $318.33
MAE: $249.52
MAPE: 19.17%


In [23]:
# Check for overfitting
print(f"\n🔍 Overfitting Check:")
print(f"R² difference (Train - Test): {train_metrics['R2'] - test_metrics['R2']:.4f}")
if abs(train_metrics['R2'] - test_metrics['R2']) < 0.05:
    print("✓ Model appears to generalize well (low overfitting)")
else:
    print("⚠️ Potential overfitting detected")


🔍 Overfitting Check:
R² difference (Train - Test): -0.0019
✓ Model appears to generalize well (low overfitting)


In [24]:
print(list(X.columns))


['Number_of_Customers_Per_Day', 'Average_Order_Value', 'Operating_Hours_Per_Day', 'Number_of_Employees', 'Marketing_Spend_Per_Day', 'Location_Foot_Traffic']


In [25]:
print("\n🔮 Example Predictions:")
print("-" * 40)

scenarios = [
    {
        'name': 'Busy Weekend Day',
        'data': {
            'Number_of_Customers_Per_Day': 180,
            'Average_Order_Value': 250,
            'Operating_Hours_Per_Day': 12,
            'Number_of_Employees': 8,
            'Marketing_Spend_Per_Day': 3000,
            'Location_Foot_Traffic': 950
        }
    },
    {
        'name': 'Slow Rainy Weekday',
        'data': {
            'Number_of_Customers_Per_Day': 60,
            'Average_Order_Value': 180,
            'Operating_Hours_Per_Day': 10,
            'Number_of_Employees': 4,
            'Marketing_Spend_Per_Day': 1000,
            'Location_Foot_Traffic': 300
        }
    },
    {
        'name': 'Average Day',
        'data': {
            'Number_of_Customers_Per_Day': 120,
            'Average_Order_Value': 200,
            'Operating_Hours_Per_Day': 11,
            'Number_of_Employees': 5,
            'Marketing_Spend_Per_Day': 2000,
            'Location_Foot_Traffic': 600
        }
    }
]

for scenario in scenarios:
    sample_df = pd.DataFrame([scenario['data']])
    sample_scaled = scaler.transform(sample_df)
    sample_selected = selector.transform(sample_scaled)   # use this only if you used SelectKBest
    prediction = model.predict(sample_selected)[0]

    print(f"\n{scenario['name']}:")
    print(f"  Predicted Revenue: ₹{prediction:.2f}")

    # Show key features
    for feature, value in scenario['data'].items():
        print(f"  {feature}: {value}")



🔮 Example Predictions:
----------------------------------------

Busy Weekend Day:
  Predicted Revenue: ₹65186.97
  Number_of_Customers_Per_Day: 180
  Average_Order_Value: 250
  Operating_Hours_Per_Day: 12
  Number_of_Employees: 8
  Marketing_Spend_Per_Day: 3000
  Location_Foot_Traffic: 950

Slow Rainy Weekday:
  Predicted Revenue: ₹44281.28
  Number_of_Customers_Per_Day: 60
  Average_Order_Value: 180
  Operating_Hours_Per_Day: 10
  Number_of_Employees: 4
  Marketing_Spend_Per_Day: 1000
  Location_Foot_Traffic: 300

Average Day:
  Predicted Revenue: ₹51077.90
  Number_of_Customers_Per_Day: 120
  Average_Order_Value: 200
  Operating_Hours_Per_Day: 11
  Number_of_Employees: 5
  Marketing_Spend_Per_Day: 2000
  Location_Foot_Traffic: 600


In [27]:
!pip install joblib



Defaulting to user installation because normal site-packages is not writeable


In [28]:
import joblib

# Save the model
joblib.dump(model, 'coffee_sales_model.pkl')

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

# Save the feature selector
joblib.dump(selector, 'feature_selector.pkl')

print("✅ Model, scaler, and selector saved successfully!")

✅ Model, scaler, and selector saved successfully!


In [29]:
!pip install streamlit 


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 1.3 MB/s eta 0:00:001.3 MB/s eta 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 3.0 MB/s eta 0:00:002.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 2.9 MB/s eta 0:00:00 MB/s eta 0:00:01:02


In [34]:
%%writefile apps.py
import streamlit as st
import pandas as pd
import joblib

# Load saved model, scaler, and selector
model = joblib.load('coffee_sales_model.pkl')
scaler = joblib.load('scaler.pkl')
selector = joblib.load('feature_selector.pkl')

st.set_page_config(page_title="Coffee Shop Revenue Predictor", page_icon="☕", layout="centered")
st.title("☕ Coffee Shop Daily Revenue Predictor")

# User inputs
num_customers = st.number_input("Number of Customers", value=100)
avg_order_value = st.number_input("Average Order Value (₹)", value=200)
operating_hours = st.number_input("Operating Hours", value=10)
num_employees = st.number_input("Number of Employees", value=5)
marketing_spend = st.number_input("Marketing Spend (₹)", value=2000)
foot_traffic = st.number_input("Location Foot Traffic", value=500)

if st.button("Predict Revenue"):
    input_df = pd.DataFrame([[num_customers, avg_order_value, operating_hours, num_employees, marketing_spend, foot_traffic]],
                            columns=['Number_of_Customers_Per_Day', 'Average_Order_Value', 'Operating_Hours_Per_Day', 'Number_of_Employees', 'Marketing_Spend_Per_Day', 'Location_Foot_Traffic'])
    scaled_input = scaler.transform(input_df)
    selected_input = selector.transform(scaled_input)
    prediction = model.predict(selected_input)[0]
    st.success(f"💰 Predicted Daily Revenue: ₹{prediction:.2f}")


Writing apps.py


In [35]:
!pwd


/home/intellact/Documents/genDs/Daily Assignment
